In [10]:
import random

import torch.nn as nn
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

manualSeed = random.randint(1, 10000)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

print("Seed:", manualSeed)

Seed: 5484


In [11]:
# Edit to View and Create More Images

batch_size = 128

# Size of Generator Input

nz = 100

# Size of Feature Maps

featureSize = 64

# Create the Datasets

transform = transforms.Compose(
    [
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

dataset_mountains = dset.ImageFolder(root="../data/mountains", transform=transform)
dataset_two = dset.ImageFolder(root="../data/mountains", transform=transform)
dataset_three = dset.ImageFolder(root="../data/mountains", transform=transform)

dataloader_mountains = torch.utils.data.DataLoader(dataset_mountains, batch_size=batch_size, shuffle=True, num_workers=2)
dataloader_two = torch.utils.data.DataLoader(dataset_two, batch_size=batch_size, shuffle=True, num_workers=2)
dataloader_three = torch.utils.data.DataLoader(dataset_three, batch_size=batch_size, shuffle=True, num_workers=2)

# Set To Zero For CPU Mode

ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu == 1) else "cpu")
    
print("Running On Device:", device)

Running On Device: cuda:0


In [12]:
# custom weights initialization called on netG and netD
def weights_init(m):
    
    classname = m.__class__.__name__
    
    if classname.find('Conv') != -1:
        
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        
    elif classname.find('BatchNorm') != -1:
        
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [13]:
class Generator(nn.Module):
    
    def __init__(self, ngpu):
        
        super(Generator, self).__init__()
        
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, featureSize * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(featureSize * 8),
            nn.ReLU(True),
            # state size. (featureSize * 8) x 4 x 4
            nn.ConvTranspose2d(featureSize * 8, featureSize * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(featureSize * 4),
            nn.ReLU(True),
            # state size. (featureSize * 4) x 8 x 8
            nn.ConvTranspose2d(featureSize * 4, featureSize * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(featureSize * 2),
            nn.ReLU(True),
            # state size. (featureSize * 2) x 16 x 16
            nn.ConvTranspose2d(featureSize * 2, featureSize, 4, 2, 1, bias=False),
            nn.BatchNorm2d(featureSize),
            nn.ReLU(True),
            # state size. (featureSize) x 32 x 32
            nn.ConvTranspose2d(featureSize, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (3) x 64 x 64
        )

    def forward(self, input):
        
        return self.main(input)

In [14]:
class Discriminator(nn.Module):
    
    def __init__(self, ngpu):
        
        super(Discriminator, self).__init__()
        
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (3) x 64 x 64
            nn.Conv2d(3, featureSize, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (featureSize) x 32 x 32
            nn.Conv2d(featureSize, featureSize * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(featureSize * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (featureSize * 2) x 16 x 16
            nn.Conv2d(featureSize * 2, featureSize * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(featureSize * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (featureSize * 4) x 8 x 8
            nn.Conv2d(featureSize * 4, featureSize * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(featureSize * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (featureSize * 8) x 4 x 4
            nn.Conv2d(featureSize * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        
        return self.main(input)

In [15]:
def main_loop(dataloader, netD, netG, epochs):
    
    criterion = nn.BCELoss()
    fixed_noise = torch.randn(64, nz, 1, 1, device=device)

    real_label = 1
    fake_label = 0

    optimizerD = torch.optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizerG = torch.optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

    # Training Loop

    img_list = []
    iters = 0

    for epoch in range(epochs):
                
        for i, data in enumerate(dataloader, 0):

            ############################
            # Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            
            # Train D on Real Images
            
            netD.zero_grad()
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), real_label, device=device)
            output = netD(real_cpu).view(-1)
            errD_real = criterion(output, label)
            errD_real.backward()

            # Generate Fake Images
            
            fake = netG(torch.randn(b_size, nz, 1, 1, device=device))
            label.fill_(fake_label)

            # Train D on Fake Images
            
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, label)
            errD_fake.backward()
            errD = errD_real + errD_fake
            optimizerD.step()

            ############################
            # Update G network: maximize log(D(G(z)))
            ###########################
            
            netG.zero_grad()
            label.fill_(real_label)
            output = netD(fake).view(-1)
            errG = criterion(output, label)
            errG.backward()
            optimizerG.step()

            if epoch % 10 == 0:
                
                print('[Epoch %d/%d] [Loss_D: %.4f] [Loss_G: %.4f]' % (epoch, epochs, errD.item(), errG.item()))

            if (iters % 500 == 0) or ((epoch == epochs - 1) and (i == len(dataloader) - 1)):
                
                with torch.no_grad():
                    
                    fake = netG(fixed_noise).detach().cpu()
                
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

            iters += 1
            
    return img_list

In [16]:
def show_results(dataloader, img_list):
    
    # Grab a batch of real images from the dataloader
    real_batch = next(iter(dataloader))

    # Plot the real images
    plt.figure(figsize=(15,15))
    plt.subplot(1,2,1)
    plt.axis("off")
    plt.title("Real Images")
    plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

    # Plot the fake images from the last epoch
    plt.subplot(1,2,2)
    plt.axis("off")
    plt.title("Fake Images")
    plt.imshow(np.transpose(img_list[-1],(1,2,0)))
    plt.show()

In [17]:
def do_run(dataloader, epochs):

    netG = Generator(ngpu).to(device)
    netG.apply(weights_init)

    netD = Discriminator(ngpu).to(device)
    netD.apply(weights_init)

    img_list = main_loop(dataloader, netG, netD, epochs)
    show_results(dataloader, img_list)

In [18]:
do_run(dataloader_mountains, 50)

RuntimeError: Given transposed=1, weight of size [100, 512, 4, 4], expected input[128, 3, 64, 64] to have 100 channels, but got 3 channels instead

In [ ]:
do_run(dataloader_two, 50)

In [ ]:
do_run(dataloader_three, 50)